This notebook summarizes and implements results from [Constructive quantum Shannon decomposition from Cartan involutions](https://iopscience.iop.org/article/10.1088/1751-8113/41/39/395305/pdf) by Byron Drury and Peter Love. There are four sections: 

In [1]:
import numpy as np
from scipy.stats import unitary_group
from scipy.linalg import expm, logm

## 1. Background

**Definition (Lie algebra):** A **Lie algebra** $A$ is a vector space with an additional operation called a Lie bracket, denoted as $[x, y]$. The Lie bracket is an alternating bilinear map $A \times A \to A$ that satisfies the Jacobi identity, i.e for $x, y, z \in A$,

$$[x, [y, z]] + [y, [z, x]] + [z, [x, y]] = 0$$



**Definition (Lie Bracket for matrix algebra):** For matrix $a, b$, the **Lie bracket** is the commutator

$$[a, b] = ab-ba$$

**Definition (Derived algebra):** The **derived algebra** of $A$, denoted $[AA]$, is the linear combinations of all brackets $[x, y]$ with $x, y \in A$

**Definition (Subalgebra):** A subspace $S \subseteq A$ is a **subalgebra** if it is closed under the Lie bracket, i.e

$$[A, A] \subseteq A $$

**Definition (Ideal):** Let $A$ be a Lie algebra. A subalgebra $I\subseteq A$ is an **ideal** if 

$$[A, I] \subseteq I$$

**Definition (Non-Abelian Lie algebra):** A Lie algebra $A$ is **non-abelian** if 

$$[AA] \neq 0$$

**Definition (Simple algebra):** A non-Abelian Lie algebra $A$ in which the only ideals are $0$ and $A$ is called **simple**. Any algebra $A$ where $[AA] = A$ is simple.

**Definition (Solvable algebra):** The derived series of an algebra $A$ is defined as 

$$A^{(0)} = A,\quad A^{(1)} = [AA], \quad A^{(2)} = [A^{(1)}A^{(1)}], \quad A^{(i)} = [A^{(i-1)}A^{(i-1)}], \ ...$$

Observe that $A^{(n+1)} \subseteq A^n$. If $A$ is finite dimensional, then the series eventually stabilizes, that is there exists an $n$ where $A^{(m)} = A^{(n)}$ for all $m\geq n$. If the series stabilizes to $0$, then $A$ is called **solvable**.

All abelian algebras are solvable, while all simple algebras are non-solvable. 

**Definition (Radical):** Every Lie algebra contains a a unique maximal solvable ideal, referred to as the **radical** of the algebra. Maximal means that no larger solvable ideal contains it. 

**Definition (Direct sum):** Let $B, C$ be subalgebra of $A$. Then, $A$ is the direct sum of $B$ and $C$, denoted $A = B \oplus C$, if $A = B+C$ and $B \cap C = 0$.

**Definition (Semi-simple algebra):** A non-zero Lie algebra $A$ is **semi-simple** if Rad $A = 0$. Equivalently, $A$ can be written as direct sum of simple subalgebra, that is 

$$A = A_1 \oplus ... \oplus A_n$$

**Definition (Real vector space):** A **real vector space** is a vector space over $\mathbb{R}$ (scalars are strictly real).

**Definition (Cartan involution):** A **Cartan involution**, denoted $\theta$, is a non-identity automorphism on the Lie algebra $\mathfrak{g}$ such that $\theta^2$ is the identity. It partitions $\mathfrak{g}$ into a direct sum of eigencespaces corresponding to the $\pm 1$ eigenvalues. So we write $\mathfrak{g} = \mathfrak{m} \oplus \mathfrak{k}$, where

$$\theta(u) = \begin{cases} u \quad &u \in \mathfrak{k} \\ -u \quad &u \in \mathfrak{m}\end{cases}$$

These two eigenspaces must both exist because of the following:

1. If only $\mathfrak{k}$ exists, then $\theta(u) = u$ for all $u\in \mathfrak{g}$, thus is $\theta$ is the identity map.

2. If only $\mathfrak{m}$ exists, then


$$\begin{align}\theta[\mathfrak{m}, \mathfrak{m}] &= [\theta(\mathfrak{m}), \theta(\mathfrak{m})] \\ &=[\mathfrak{m}, \mathfrak{m}] \\ &\subseteq \mathfrak{k}\end{align}$$

The second equality results from the bilinearity of the Lie bracket and the third equality shows that $[\mathfrak{m}, \mathfrak{m}]$ has $+1$ eigenvalue. This shows that $\mathfrak{m}$ cannot be a subalgebra.

The Cartan involution directly leads to the Cartan decomposition discussed below.

**Theorem (Cartan decomposition on Lie algebra):** A **Cartan decomposition** of a real semi-simple Lie algebra $\mathfrak g$ is a decomposition $\mathfrak{g} = \mathfrak{k} \oplus \mathfrak{m}$ where $\mathfrak{m} = \mathfrak{k}^\perp$, for which $\mathfrak{m}$ and $\mathfrak{k}$ statisfy the commutation relations

$$[\mathfrak{k}, \mathfrak{k}] \subseteq \mathfrak{k}, \quad [\mathfrak{m}, \mathfrak{k}] = \mathfrak{m}, \quad [\mathfrak{m}, \mathfrak{m}] \subseteq \mathfrak{k}$$

Since $\mathfrak{k}$ is closed under the Lie bracket, it is a subalgebra of $\mathfrak{g}$. Suppose $\mathfrak{h}$ is a subalgebra of $\mathfrak{m}$, we have $[\mathfrak{h}, \mathfrak{h}] \subseteq \mathfrak{h} \subseteq \mathfrak{m}$ and $ \mathfrak{m} \cap \mathfrak{k} = 0$ implies that 

$$ [\mathfrak{h}, \mathfrak{h}] = 0$$

Thus all subalgebra of $\mathfrak{m}$ is abelian. The maximal subalgebra is called the Cartan subalgebra.

**Definition (Adjoint action):** The Lie group $G$ acts on its Lie algebra $\mathfrak{g}$ through a conjugation, known as the **adjoint action**, $Ad_G: \mathfrak{g} \to \mathfrak{g}$ defined by 

$$Ad_u(X) = u^\dagger X u$$

for $u \in G$ and $X \in \mathfrak{g}$.

**Definition (Adjoint orbit):** For $K = \exp(\mathfrak{k})$, the **adjoint orbit** of X is defined to be

$$Ad_K X = \bigcup_{k\in K} Ad_k X$$

**Theorem ($\mathfrak{h}$-adjoint theorem):** For any two maximal Abelian subalgebras $\mathfrak{h}$ and $\mathfrak{h}^\prime$ in $\mathfrak{m}$, there is an element $k\in K$ such that $Ad_k(\mathfrak{h}) = \mathfrak{h}^\prime$. Furthermore, the adjoint orbit of $\mathfrak{h}$ is equal to $\mathfrak{m}$, i.e.

$$\mathfrak{m} = \bigcup_{k\in K} Ad_k \mathfrak{h}$$

($\subseteq$) All elements of $\mathfrak{m}$ is in a maximal Abelian subalgebra of $\mathfrak{m}$

($\supseteq$) All subalgebra adjoint to one a maximal Abelian subalgebra of $\mathfrak{m}$ is included in $\mathfrak{m}$

**Theorem (Global Cartan decomposition):** For a semi-simple Lie algebra $\mathfrak{g}$ with Cartan decomposition $\mathfrak{g} = \mathfrak{k} \oplus \mathfrak{m}$ and a Cartan subalgebra $\mathfrak{m}$, let $A = \exp(\mathfrak{h})$ and $K = \exp(\mathfrak{k})$, then 

$$G=K A K$$

is called the global Cartan decomposition of the Lie group $G = \exp(\mathfrak{g})$.

**Proof.** Taking the exponential map of the Cartan decomposition, we have

\begin{align}G &= \exp(\mathfrak{g}) \\ &= \exp(\mathfrak{m} \oplus \mathfrak{k}) \\ &= \exp(\mathfrak{m})\exp(\mathfrak{k}) \\ &= KM\end{align}

Let $\mathfrak{h}$ be a maximal abelian subalgebra of $\mathfrak{m}$. By the adjoint orbit theorem, 

\begin{align} \exp(\mathfrak{m}) &= \bigcup_{k\in K} \exp(k^\dagger \mathfrak{h} k) \\ &= \bigcup_{k\in K}  k^\dagger \exp(\mathfrak{m}) k \\ &= K^\dagger A K \end{align}


Since $\mathfrak{k}$ is a subspace, $K$ is closed under multiplication. Substituting into the original decomposition

\begin{align} G &= KM \\ &= K(K^\dagger A K) \\ &= KAK \quad\quad\quad\quad\square\end{align}  



**Definition (Global Cartan involution):** The **global Cartan involution** $\Theta$ has the following properties on the Lie group $G = \exp(\mathfrak{g})$

$$\Theta(U) = \begin{cases}U \quad &U \in \exp(\mathfrak{k}) \\ U^\dagger \quad &U \in \exp(\mathfrak{m}) \end{cases}$$

For unitary group, whose elements satisfy the condition $U^{-1} = U^\dagger$, the global Cartan involution is given by

$$\Theta(\exp(\mathfrak{g})) = \exp(\theta(\mathfrak{g}))$$

In the case of $\mathfrak{su}(n)$, there are only three classes of Cartan decomposition, denoted **AI**, **AII**, **AIII**. The $\mathfrak{k}$ subalgebras of $\mathfrak{su}(n)$ are isomorphic to $\mathfrak{so}(n)$, $\mathfrak{sp}(\frac{n}{2})$ and $\mathfrak{s}[\mathfrak{u}(p) + \mathfrak{u}(q)]$ for any $p+q=n$ ($n$ even) for **AI**, **AII**, **AIII** decompositions, respectively.

**Remark ($\mathfrak{k}$-subalgebra of $\mathfrak{su}(n)$):** 

| Cartan Type | $\mathfrak{k}$-Subalgebra | Properties | $K$-Subgroup | Properties | Dimension |
| :- | :- | :- | :- | :- | :- |
| **AI** | $\mathfrak{so}(n)$ | Real Skew-Symmetric | $SO(n)$ | Special Orthogonal | $\frac{n(n-1)}{2}$ |
| **AII** | $\mathfrak{sp}\big(\frac{n}{2}\big)$ | ... | $Sp\big(\frac{n}{2}\big)$ | Sympletic | ... |
| **AIII** | $\mathfrak{s}[\mathfrak{u}(p)$$\oplus\mathfrak{u}(q)]$ | Traceless Block Skew-Hermitian | $S[U(p)$$\oplus$$U(q)]$ | Special Block Unitary | $p^2$$+ q^2$$- 1$ |
| **A*** | $\mathfrak{su}(n)$ | Complex Traceless Skew-Hermitian | $SU(n)$ | Special Unitary | $n^2-1$ |

Observe that $\mathfrak{k}$-subalgebra of Type AI Cartan involution satisfies 

$$u \in \mathfrak{k}_{\mathbf{AI}} \iff \theta_{\mathbf{AI}}(u) = -u^T = u \iff u^T = -u \iff u\text{ is skew-symmetric} \iff u \in \mathfrak{so}(n)$$

**Theorem ($M$-factoring theorem):** For any $U \in SU(2^n)$, with the Cartan decomposition $U = KM$, $M^2$ is uniquely determined by

$$M^2 = \Theta(U^\dagger)U$$

**Proof.**

\begin{align}\Theta(G^\dagger)G &= \Theta(M^\dagger K^\dagger)KM \\ &= \Theta (M^\dagger) \Theta(K^\dagger) KM \\ &= (MK^\dagger)K M \\ &= M^2\quad\quad\quad\quad\square\end{align}

## 2. Single-Qubit Unitary

The Lie algebra $\mathfrak{su}(2)$ is given by

$$\mathfrak{su}(2) = \text{span}_\mathbb{R}i\{X, Y, Z\}$$

For choice of Cartan involution, we have a different Cartan decomposition.

### Type AI

For $u \in \mathfrak{su}(2)$, type **AI** Cartan involution is given by 

$$\theta(u) = -u^T$$

Applying $\theta$ on each of the basis, we get the Cartan decomposition

$$\mathfrak{k} = \text{span}_\mathbb{R} i \{Y\}, \quad \mathfrak{m} = \text{span}_\mathbb{R} i\{X, Z\} $$

Furthermore, each of $\text{span}_\mathbb{R} i \{X\}$ and $\text{span}_\mathbb{R} i \{Y\}$ is a maximal abelian subalgebra of $\mathfrak{su}(2)$ contained in $\mathfrak{m}$. They are related through the adjoint action.

$$\text{span}_\mathbb{R} i \{Z\} = e^{i\frac{\pi}{4}Y} \cdot \text{span}_\mathbb{R} i \{X\} \cdot e^{-i\frac{\pi}{4}Y} $$

We choose $\mathfrak{h} = \text{span}_\mathbb{R} i \{Z\}$

Taking the exponential, we get type **AI** global Cartan involution

\begin{align}\Theta(U) &= \exp(\theta(u)) \\ &= \exp(-u^T) \\ &= (U^\dagger)^T \\ &= U^\ast \end{align}

### Type AIII

For $u \in \mathfrak{su}(2)$, type **AIII** Cartan involution is given by 

$$\theta(u) = YuY$$

Applying $\theta$ on each of the basis, we get the same Cartan decomposition (as above)

Taking the exponential, we get type **AIII** global Cartan involution

$$\Theta(U) = YUY$$

### Procedure

**Step 1.** We use $M$-factoring theorem to calculate $\Theta(U^\dagger)U = M^2 $

**Step 2.** Diagonalize $M^2 = PDP^\dagger$. Since $D$ is diagonal, $D$ must be of the form $e^{i\alpha Z}$, that is $D \in \exp(\mathfrak{h})$. Furthermore, by $\mathfrak{h}$-adjoint theorem, $P \in \exp(\mathfrak{k})$.

**Step 3.** We have $M = PD^{1/2}P^\dagger$, thus $A = D^{1/2}$ and $K_2 = P^\dagger$. So 

$$U = K^\prime M = K^\prime (K_2^\dagger A K_2)$$

Then $K^\prime = UM^\dagger$. Thus $K_1 = K^\prime K_2^\dagger$

**Step 4.** We want to decompose $K_1, A, K_2$ into $e^{i\alpha Y}, e^{i\beta Z}, e^{i\gamma Y}$ respectively. This corresponds to finding $\alpha, \beta, \gamma$. We have the following closed form for the exponential of $Y$ and $Z$.

$$e^{i\theta Y} = \begin{pmatrix} \cos\theta & \sin\theta \\ -\sin\theta & \cos\theta \end{pmatrix}, \quad e^{i\theta Z} = \begin{pmatrix} e^{i\theta} & 0 \\ 0 & e^{-i\theta} \end{pmatrix}$$

#### YZY decomposition

We can generate a random element $U \in U(2)$ but we want it to be unimodular. 

So we let $U^\prime = aU$. To find $a$, note that 

$$\det(aU) = a^m det(U)$$

where $m = \dim(U)$. We want that $a^m = \frac{1}{\det(U)}$. So $a = \det(U)^{-\frac{1}{m}}$.

In [2]:
# Helper functions

# Define dagger
dagger = lambda U: np.conj(U).T

# Pauli matrices
I = np.identity(2)
X = np.array([[0, 1], [1, 0]])
Y = np.array([[0, -1j], [1j, 0]])
Z = np.array([[1, 0], [0, -1]])

In [3]:
# Define Type AI Global Cartan involution
AI_Theta = lambda U: np.conj(U)

# Define Type AIII Global Cartan involution
AIII_Theta = lambda U: Y@U@Y

In [4]:
# Generate Random SU(2)
U = unitary_group.rvs(2)
U /= np.sqrt(np.linalg.det(U)) # Normalize the determinant

In [5]:
# Step 1: Theta(Udagger)U = M^2
M_squared = AIII_Theta(dagger(U))@U

# Step 2: Diagonalize M^2
D, P = np.linalg.eig(M_squared)
D = np.diag(D)

# Step 3: A = sqrt(D), K2 = Pdagger, Kprime = UMdagger, K1 = Kprime K2dagger
A = np.sqrt(D)
K2 = dagger(P)

M = P@A@dagger(P)
K_prime = U@dagger(M)

K1 = K_prime @ dagger(K2)

print(f"Equality Check: {np.allclose(K1@A@K2, U)}")

Equality Check: True


In [6]:
# Extract angle using arctan2 to preserve quadrant
alpha = np.arctan2(np.real(K1[0, 1]), np.real(K1[1, 1]))
gamma = np.arctan2(np.real(K2[0, 1]), np.real(K2[1, 1]))

# Extract angle of the complex number
beta = np.angle(A[0, 0])

In [7]:
# Verify decomposition
print(f"K1: {np.allclose(expm(1j*alpha*Y), K1)}")
print(f"A: {np.allclose(expm(1j*beta*Z), A)}")
print(f"K2: {np.allclose(expm(1j*gamma*Y), K2)}")
print(f"U: {np.allclose(expm(1j*alpha*Y)@expm(1j*beta*Z)@expm(1j*gamma*Y), U)}")

K1: True
A: True
K2: True
U: True


#### X+Y+Z decomposition

In [8]:
# Matrix logarithm for x+y+z decomposition
# this is a complex traceless skew-hermitian matrix
u = np.round(logm(U), 14)

In [9]:
# Extract parameters, these must be real
ax = np.real(u[1, 0]/1j)
ay = np.imag(u[1, 0]/1j)
az = u[0, 0]/1j

In [10]:
# Verify decomposition
print(f"U: {np.allclose(expm(1j*(ax*X + ay*Y + az*Z)), U)}")

U: True


## 3. Two-Qubits Unitary

The Lie algebra $\mathfrak{su}(4)$ is given by

$$\mathfrak{su}(4) = \text{span}_\mathbb{R}i \{ \text{IX, IY, IZ}, \mathbf{XI}, \mathbf{XX}, \mathbf{XY}, \mathbf{XZ}, \text{YI, YX, YY, YZ}, \mathbf{ZI}, \mathbf{ZX}, \mathbf{ZY}, \mathbf{ZZ} \}$$

Note that $\text{I}^{\otimes 2}$ is not included because the its exponential is the global phase. 

For reasons that will become clear, we choose type **AI** Cartan involution.

For $u \in \mathfrak{su}(4)$, type **AI** Cartan involution is given by

$$\theta(u) = -u^T$$

with its global Cartan involution being $\Theta(U) = U^\ast$.

Observe the action of $\theta$ on $\mathfrak{k}$ and $\mathfrak{m}$ subspaces.

$$\theta(\mathfrak{k}) = -\mathfrak{k}^T = \mathfrak{k}, \quad \theta(\mathfrak{m}) = -\mathfrak{m}^T = -\mathfrak{m}$$

It follows that $\mathfrak{k}^T = -\mathfrak{k}$ and $\mathfrak{m}^T = \mathfrak{m}$. Thus $\mathfrak{k}$ is skew-symmetric and $\mathfrak{m}$ is symmetric. Then, one can check that

$$\mathfrak{k} = \text{span}_\mathbb{R}i\{\text{IY, XY, ZY}, \mathbf{YI}, \mathbf{YX}, \mathbf{YZ}\}, \quad \mathfrak{m} = \text{span}_\mathbb{R}i\{\text{IX, IY, IZ}, \mathbf{XI}, \mathbf{XX}, \mathbf{XZ}, \text{ZI, ZX, ZZ}, \mathbf{YY}\}$$

We choose our Cartan subalgebra to be $\mathfrak{h} = \text{span}_\mathbb{R}i\{\text{IZ, ZI, ZZ}\}$.

**Step 1.** Compute $M^2 = \Theta(U^\dagger)U = U^T U$

**Step 2.** Diagonalize $M^2 = PDP^\dagger$. Since $D$ is diagonal, we have that $D \in \exp(\mathfrak{h})$. Furthermore, by $\mathfrak{h}$-adjoint theorem, $P \in \exp(\mathfrak{k})$.

**Step 3.** We have $M = PD^{1/2}P^\dagger$, thus $A = D^{1/2}$ and $K_2 = P^\dagger$. So 

$$U = K^\prime M = K^\prime (K_2^\dagger A K_2)$$

Then $K^\prime = UM^\dagger$. Thus $K_1 = K^\prime K_2^\dagger$

**Step 4.** Observe that the dimension of $SO(4)$ and $SU(2) \otimes SU(2)$ coincides. As such, there is a homomorphism between the two Lie algebra (and Lie group) by the conjugation of the "magic basis"

$$Ad_B (\mathfrak{so}(4)) = \mathfrak{su}(2) \oplus \mathfrak{su}(2)$$

where $B$ is the matrix (up to switching columns)

$$B = \frac{1}{\sqrt2}
\begin{pmatrix} 
1 & 0 & 0 & i \\
0 & i & 1 & 0 \\
0 & i & -1 & 0 \\
1 & 0 & 0 & -i
\end{pmatrix}$$

Importantly, $Ad_B$ maps $\mathfrak{h}$ to another Cartan subalgebra by

$$Ad_B(\text{IZ}) = -\text{YY}, \quad Ad_B(\text{ZI}) = \text{XX}, \quad Ad_B(\text{ZZ}) = \text{ZZ}$$

We can also check the $\mathfrak{k}$-subalgebra homomorphism.

\begin{align}
Ad_B(\text{IY}) = -\text{IX}, \quad Ad_B(\text{XY}) = -\text{ZI}, \quad Ad_B(\text{ZY}) = -\text{XI} \\
Ad_B(\text{YI}) = -\text{YI}, \quad Ad_B(\text{YX}) = -\text{IZ}, \quad Ad_B(\text{YZ}) = \text{IY} \\
\end{align}

In [11]:
# Helper Functions
def decompose_one_qubit_product(
    U: np.ndarray, validate_input: bool = True, atol: float = 1e-8, rtol: float = 1e-5
):
    """
    Decompose a 4x4 unitary matrix to two 2x2 unitary matrices.
    Args:
        U (np.ndarray): input 4x4 unitary matrix to decompose.
        validate_input (bool): if check input.
    Returns:
        phase (float): global phase.
        U1 (np.ndarray): decomposed unitary matrix U1.
        U2 (np.ndarray): decomposed unitary matrix U2.
        atol (float): absolute tolerance of loss.
        rtol (float): relative tolerance of loss.
    Raises:
        AssertionError: if the input is not a 4x4 unitary or
        cannot be decomposed.
    """

    """if validate_input:
        assert np.allclose(
            makhlin_invariants(U, atol=atol, rtol=rtol), (1, 0, 3), atol=atol, rtol=rtol
        )"""

    i, j = np.unravel_index(np.argmax(U, axis=None), U.shape)

    def u1_set(i):
        return (1, 3) if i % 2 else (0, 2)

    def u2_set(i):
        return (0, 1) if i < 2 else (2, 3)

    u1 = U[np.ix_(u1_set(i), u1_set(j))]
    u2 = U[np.ix_(u2_set(i), u2_set(j))]

    u1 = to_su(u1)
    u2 = to_su(u2)

    phase = U[i, j] / (u1[i // 2, j // 2] * u2[i % 2, j % 2])

    return phase, u1, u2

def to_su(u: np.ndarray) -> np.ndarray:
    """
    Given a unitary in U(N), return the
    unitary in SU(N).
    Args:
        u (np.ndarray): The unitary in U(N).
    Returns:
        np.ndarray: The unitary in SU(N)
    """

    return u * complex(np.linalg.det(u)) ** (-1 / np.shape(u)[0])

In [12]:
from numpy.linalg import det

In [13]:
# Magic Basis
B = 1/np.sqrt(2) * np.array([[1, 0, 0, 1j], 
                             [0, 1j, 1, 0], 
                             [0, 1j, -1, 0], 
                             [1, 0, 0, -1j]])

Line 15-18, 22-24 that's it!

In [26]:
# Generate Random SU(4)
U = unitary_group.rvs(4, random_state = 18)

U = U/ np.linalg.det(U)**0.25  # Normalize the determinant

U_prime = dagger(B) @ U @ B

# Step 1 M^2=U^TU
M_squared = U_prime.T @ U_prime

M_squared = np.round(M_squared, 14)

# Step 2 M^2 = PDPdagger
D, P = np.linalg.eig(M_squared)
D = np.diag(D)

if np.isclose(np.linalg.det(P), -1):
    P[:, [0, 1]] = P[:, [1, 0]]  # det(P) = 1
    D[[0, 1]] = D[[1, 0]] 
    D[:, [0, 1]] = D[:, [1, 0]] # det(D) = 1

# Step 3 A = sqrtD, K2 = Pdagger, M = P A Pdagger,
# Kprime = UMdagger, K1 = Kprime K2dagger
A = np.sqrt(D)
if np.isclose(np.linalg.det(A), -1):
    A[0, 0] *= - 1 # det(A) = 1

K2 = dagger(P)

M = P @ A @ dagger(P)

K = U_prime @ dagger(M)

K1 = K @ dagger(K2)

assert np.allclose(K1, np.real(K1)), "K1 not orthogonal"
assert np.allclose(K2, np.real(K2)), "K2 not orthogonal"
assert np.isclose(np.linalg.det(A), 1), "det(A) not 1"
assert np.isclose(np.linalg.det(K1), 1), "det(K1) not 1"
assert np.isclose(np.linalg.det(K2), 1), "det(K2) not 1"
assert np.allclose(K1@A@K2, U_prime), "KAK not equal U"

# Step 4 Extract the parameters
# Taking the elements of the maximal torus
# Reducing it back to Lie algebra form
# Since the torus has rank 3, we can ignore the last angle
torus = np.angle(np.diag(A))[:3]

# The coefficient matrix of the maximal torus is given by
coef_mat = np.array([[1, 1, 1], [-1, 1, -1], [1, -1, -1]])

# Multiply the inverse with the torus to give the torus parameter
a, b, c = np.linalg.inv(coef_mat) @ torus

# Multiply it with the appropriate IZ, ZI, ZZ terms
assert np.allclose(
    expm(1j * (a * np.kron(I, Z) + b * np.kron(Z, I) + c * np.kron(Z, Z))), A), "IZ, ZI, ZZ"

yy_term, xx_term, zz_term = -a, b, c # Unpack xx, yy, zz parameter

L = B@K1@dagger(B) # Left local term
R = B@K2@dagger(B) # Right local term
CAN = B@A@dagger(B) # XX+YY+ZZ term

# Check Canonical Representation
assert np.allclose(
    expm(1j * (xx_term * np.kron(X, X) + yy_term * np.kron(Y, Y) +
               zz_term * np.kron(Z, Z))), CAN), "XX, YY, ZZ"

phase1, l1, l2 = decompose_one_qubit_product(L) # Decompose into SU(2) x SU(2)
phase2, r1, r2 = decompose_one_qubit_product(R) # Decompose into SU(2) x SU(2)

if not np.isclose(phase1, 1) and not np.isclose(phase2, 1):
    pboth +=1

assert np.allclose(phase1*np.kron(l1, l2), L), "L"
assert np.allclose(phase2*np.kron(r1, r2), R), "R"

assert np.allclose(phase1*phase2*np.kron(l1, l2)@expm(1j * (xx_term * np.kron(X, X) + yy_term * np.kron(Y, Y) +
               zz_term * np.kron(Z, Z)))@np.kron(r1, r2), U), "Uh oh..."

In [44]:
xx_term, yy_term, zz_term

(1.3692494801887862, -0.7426011592249738, 0.5892432100321542)